# EXAMPLES (RAG)
- [RAG](https://docs.activeloop.ai/examples/rag)
  - [RAG Quickstart](https://docs.activeloop.ai/examples/rag/quickstart)
  - [RAG Tutorials](https://docs.activeloop.ai/examples/rag/tutorials)
    - [Vector Store Basics](https://docs.activeloop.ai/examples/rag/tutorials/vector-store-basics)
    - [Vector Search Options](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options)
      - [LangChain API](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options/langchain-api)
      - [Deep Lake Vector Store API](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options/vector-store-api)
      - [**Managed Database REST API**](https://docs.activeloop.ai/examples/rag/tutorials/vector-search-options/rest-api)
    - [Customizing Your Vector Store](https://docs.activeloop.ai/examples/rag/tutorials/step-4-customizing-vector-stores)
    - [Image Similarity Search](https://docs.activeloop.ai/examples/rag/tutorials/image-similarity-search)
    - [Improving Search Accuracy using Deep Memory](https://docs.activeloop.ai/examples/rag/tutorials/deepmemory)


## RAG Tutorials (Vector Search Options) (Managed Database REST API)

### Performing Vector Search Using the REST API

In [1]:
# Query this Vector Store stored in the Managed Tensor Database using the REST API. The steps are:
# - Define the authentication tokens and search terms
# - Embed the search search term using OpenAI
# - Reformat the embedding to an embedding_search string that can be passed to the REST API request.
# - Create the query string using Deep Lake TQL. The dataset_path and embedding_search are a part of the query string.  
# - Submit the request and print the response data data

In [2]:
import requests
import openai
import os
from dotenv import load_dotenv

load_dotenv(override = True)
open_api_key = os.getenv('OPENAI_API_KEY')
activeloop_token = os.getenv('ACTIVELOOP_TOKEN')

MODEL_GPT = 'gpt-4o-mini'

In [3]:
# Tokens should be set in environmental variables.
ACTIVELOOP_TOKEN = os.environ['ACTIVELOOP_TOKEN']
DATASET_PATH = 'hub://activeloop/twitter-algorithm'
ENDPOINT_URL = 'https://app.activeloop.ai/api/query/v1'
SEARCH_TERM = 'What do the trust and safety models do?'
# os.environ['OPENAI_API_KEY'] OPEN AI TOKEN should also exist in env variables

In [4]:
# The headers contains the user token
headers = {
    "Authorization": f"Bearer {ACTIVELOOP_TOKEN}",
}

In [5]:
# Embed the search term

# embedding = openai.Embedding.create(input=SEARCH_TERM, model="text-embedding-ada-002")["data"][0]["embedding"]
embedding = openai.embeddings.create(input=SEARCH_TERM, model="text-embedding-ada-002").data[0].embedding

In [6]:
# Format the embedding array or list as a string, so it can be passed in the REST API request.
embedding_string = ",".join([str(item) for item in embedding])

# Create the query using TQL
# query = f"select * from (select text, cosine_similarity(embedding, ARRAY[{embedding_string}]) as score from \"{dataset_path}\") order by score desc limit 5"
query = f"select * from (select text, cosine_similarity(embedding, ARRAY[{embedding_string}]) as score from \"{DATASET_PATH}\") order by score desc limit 5"
          
# Submit the request
response = requests.post(ENDPOINT_URL, json={"query": query}, headers=headers)

data = response.json()

In [7]:
# print(data)
print(data["description"])
print(data["tensors"])
# print(data["data"])

Query successful.
['text', 'score']


In [8]:
print(data["data"][0]["score"])

0.8365592956542969


In [9]:
print(data["data"][0]["text"])

Trust and Safety Models

We decided to open source the training code of the following models:
- pNSFWMedia: Model to detect tweets with NSFW images. This includes adult and porn content.
- pNSFWText: Model to detect tweets with NSFW text, adult/sexual topics.
- pToxicity: Model to detect toxic tweets. Toxicity includes marginal content like insults and certain types of harassment. Toxic content does not violate Twitter's terms of service.
- pAbuse: Model to detect abusive content. This includes violations of Twitter's terms of service, including hate speech, targeted harassment and abusive behavior.

We have several more models and rules that we are not going to open source at this time because of the adversarial nature of this area. The team is considering open sourcing more models going forward and will keep the community posted accordingly.


In [10]:
print(data)

{'description': 'Query successful.', 'tensors': ['text', 'score'], 'data': [{'text': "Trust and Safety Models\n=======================\n\nWe decided to open source the training code of the following models:\n- pNSFWMedia: Model to detect tweets with NSFW images. This includes adult and porn content.\n- pNSFWText: Model to detect tweets with NSFW text, adult/sexual topics.\n- pToxicity: Model to detect toxic tweets. Toxicity includes marginal content like insults and certain types of harassment. Toxic content does not violate Twitter's terms of service.\n- pAbuse: Model to detect abusive content. This includes violations of Twitter's terms of service, including hate speech, targeted harassment and abusive behavior.\n\nWe have several more models and rules that we are not going to open source at this time because of the adversarial nature of this area. The team is considering open sourcing more models going forward and will keep the community posted accordingly.", 'score': 0.836559295654